In [ ]:
import requests
import pandas as pd
from datetime import datetime, timedelta
import time

def get_funding_rates(symbol="SOLUSDT", interval_hours=8, days=5*365):
    endpoint = "https://fapi.binance.com/fapi/v1/fundingRate"
    start_time = datetime.utcnow() - timedelta(days=days)
    start_ts = int(start_time.timestamp() * 1000)

    all_data = []
    while start_ts < int(datetime.utcnow().timestamp() * 1000):
        params = {
            "symbol": symbol,
            "startTime": start_ts,
            "limit": 1000
        }
        response = requests.get(endpoint, params=params)
        data = response.json()

        if not data:
            break

        all_data.extend(data)

        # Update start time for next batch
        start_ts = data[-1]['fundingTime'] + 1

        # Respect API rate limit
        time.sleep(0.5)

    # Convert to DataFrame
    df = pd.DataFrame(all_data)
    df['fundingTime'] = pd.to_datetime(df['fundingTime'], unit='ms')
    df['fundingRate'] = pd.to_numeric(df['fundingRate'], errors='coerce')

    # Resample to daily by summing 3 funding rates per day
    df.set_index('fundingTime', inplace=True)
    df_daily = df['fundingRate'].resample('1D').sum().dropna()
    df_daily.name = 'daily_funding_rate'

    return df_daily.to_frame()

df_funding_sol = get_funding_rates()
print(df_funding_sol.head())



/Users/hamzamuhammad/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:34: NotOpenSSLWarning: urllib3 v2.0 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


             daily_funding_rate
fundingTime                    
2020-09-13               0.0001
2020-09-14               0.0003
2020-09-15               0.0003
2020-09-16               0.0003
2020-09-17               0.0003


In [2]:
df_funding_sol.to_excel("sol_funding.xlsx")